In [0]:
!pip3 install pulp

In [0]:
import pulp
import numpy as np
from scipy.spatial import distance

## Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP) - Model 1

Paper Link - [Vehicle Routing Problem with Deliveries and Pickups: Modelling Issues and Meta-heuristics Solution Approaches](https://core.ac.uk/download/pdf/19477982.pdf)

<div align="justify">
The code provided below creates a <code>class</code> to frame and solve the Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP) using the two-index flow formulation as mentioned in the paper. The formulation of the problem as a Mixed ILP is as follows:
</div>

$$
min \sum_{i=0}^{n} \sum_{j=0}^{n} d_{ij} x_{ij}
$$

$$
s.t. \sum_{i=0}^{n} x_{ij} = 1, j \in \{1, ..., n\}, - (3.1)
$$

$$
\sum_{i=0}^{n} x_{ji} = 1, j \in \{1, ..., n\}, - (3.2)
$$

$$
\sum_{i=0}^{n} R_{ij} - q_{j} = \sum_{i=0}^{n} R_{ji}, j \in \{1, ..., n\}, - (3.3)
$$

$$
\sum_{i=0}^{n} P_{ij} + b_{j} = \sum_{i=0}^{n} P_{ji}, j \in \{1, ..., n\}, - (3.4)
$$

$$
\sum_{i=1}^{n} R_{i0} = 0, - (3.5)
$$

$$
\sum_{i=1}^{n} P_{0i} = 0, - (3.6)
$$

$$
R_{ij} + P_{ij} \leq C x_{ij}, i, j \in \{0, ..., n\}, - (3.7)
$$

$$
x_{ij} = \{0, 1\}, i, j \in \{1, ... n\}
$$

$$
R_{ij}, P_{ij} \geq 0, i, j \in \{1, ..., n\}
$$

where,<br />

d<sub>ij</sub> = Cost to travel from node i to j.<br />
q<sub>i</sub> = The delivery demand of node i.<br />
b<sub>i</sub> = The pickup demand of node i.<br />
C = Vehicle capacity.<br />

<b>Decision Variables:</b>
1. x<sub>ij</sub> = {<b>1</b>, if edge (i, j) present in any route. <b>0</b>, otherwise}.
2. R<sub>ij</sub> = The amount of delivery goods on board on arc ij.
3. P<sub>ij</sub> = The amount of pickup goods on board on arc ij.

<div align="justify">
The objective is to minimize the sum of the distances of all routes that satisfy the given constraints. The constraints (3.1) and (3.2) ensure that every node is visited exactly once (every node leads to only one other node, every node must be visited by only one other node). The constraints (3.3) and (3.4) ensure that the flow conservation is met (at a node <i>j</i>, the amount of delivery load after servicing delivery of node <i>j</i>, must be equal to pickup load at the same node. The pickup load condition is similar in nature). The constraints (3.5) and (3.6) ensure that the depot has zero pickup and delivery load. The constraint (3.7) ensures that the pickup and demand loads do not exceed the capacity of the vehicle at all nodes in the routes.  
</div>

## Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP) - Model 2

Book - [Vehicle Routing Problems, Methods, and Applications, 2nd Edition by Paolo Toth and Danielle Virgo (pages: 167-168)](https://my.siam.org/Store/Product/viewproduct/?ProductId=26037961)

<div align="justify">
The code provided below creates a <code>class</code> to frame and solve the Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP) using the three-index flow formulation as mentioned in the paper. The formulation of the problem as a Mixed ILP is as follows:
</div>

ToDo: Constraints

<div align="justify">
The objective is to minimize the sum of the distances of all routes that satisfy the given constraints. Constraints (4.1) and (4.2) ensure that each customer is visited exactly once (each node is leads to one other node and is exclusive the vehicle <i>v</i>, ). Constraint (4.3) ensures that each vehicle is not reused for another route. Constraint (4.4) ensures that the delivery and pickup loads do not exceed the vehicle capacity for each vehicle at each node in the route. Constraint (4.5) and (4.6) ensure that the flow conservation is met (at a node <i>i</i>, the amount of delivery load after servicing delivery of node <i>i</i>, must be equal to pickup load at the same node. The pickup load condition is similar in nature). 
</div>


In [0]:
class VRPSDP:
    def __init__(self, costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle):
        self.costMatrix = costMatrix
        self.n = len(costMatrix)
        self.demand = demand
        self.pickup = pickup
        self.numberOfVehicles = numberOfVehicles
        self.capacityOfVehicle = capacityOfVehicle
        self.initialzeLP()

    def initialzeLP(self):
        self.lp = pulp.LpProblem('VRP-SDP', pulp.LpMinimize)
        x, R, P = [], [], []

        # Create decision variables
        for i in range(self.n):
            xRow, RRow, PRow = [], [], []
            for j in range(self.n):
                xRow.append(pulp.LpVariable('x('+str(i)+","+str(j)+")", cat='Binary'))
                RRow.append(pulp.LpVariable('R('+str(i)+","+str(j)+")", lowBound=0, cat='Continuous'))
                PRow.append(pulp.LpVariable('P('+str(i)+","+str(j)+")", lowBound=0, cat='Continuous'))
            x.append(xRow)
            R.append(RRow)
            P.append(PRow)

        # Create objective
        objective = None
        for i in range(self.n):
            for j in range(self.n):
                objective += self.costMatrix[i][j] * x[i][j]
        self.lp += objective

        # constraint 1
        for j in range(1, self.n):
            const1 = None
            for i in range(self.n):
                const1 += x[i][j]
            self.lp += const1 == 1
        
        # constraint 2
        for j in range(1, self.n):
            const2 = None
            for i in range(self.n):
                const2 += x[j][i]
            self.lp += const2 == 1

        # constraint 3
        for j in range(1, self.n):
            const3a, const3b = None, None
            for i in range(self.n):
                const3a += R[i][j]
                const3b += R[j][i]
            self.lp += const3a - self.demand[j] == const3b

        # constraint 4
        for j in range(1, self.n):
            const4a, const4b = None, None
            for i in range(self.n):
                const4a += P[i][j]
                const4b += P[j][i]
            self.lp += const4a + self.pickup[j] == const4b

        # constraint 5
        const5 = None
        for i in range(1, self.n):
            const5 += P[0][i]
        self.lp += const5 == 0

        # constraint 6
        const6 = None
        for i in range(1, self.n):
            const6 += R[i][0]
        self.lp += const6 == 0

        # constraint 7
        for i in range(self.n):
            for j in range(self.n):
                self.lp += R[i][j] + P[i][j] <= self.capacityOfVehicle * x[i][j]

        # constraint 8
        for i in range(1, self.n):
            self.lp += x[0][i] <= self.numberOfVehicles

    def solve(self):
        status = self.lp.solve()
        print(pulp.LpStatus[status])
    
    def getResult(self):
        print("Objective value: ", pulp.value(self.lp.objective))
        for v in self.lp.variables():
            print(v.name, " = ", v.varValue)

## E-N13-K4.vrp

In [0]:
costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],   
[9,0,21,22,36,52,24,51,36,37,41,30,20],    
[14,21,0,25,38,5,31,7,36,43,29,7,6],    
[23,22,25,0,42,12,35,17,44,31,31,11,6],
[32,36,38,42,0,22,37,16,46,37,29,13,14],   
[50,52,5,12,22,0,41,23,10,39,9,17,16],   
[21,24,31,35,37,41,0,26,21,19,10,25,12],  
[49,51,7,17,16,23,26,0,30,28,16,27,12],   
[30,36,36,44,46,10,21,30,0,25,22,10,20],    
[27,37,43,31,37,39,19,28,25,0,20,16,8],   
[35,41,29,31,29,9,10,16,22,20,0,10,10],   
[28,30,7,11,13,17,25,27,10,16,10,0,10],
[18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

len(costMatrix)

13

In [0]:
demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100] 
pickup = [1100, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
capacityOfVehicle = 6000
numberOfVehicles = 4

In [0]:
lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve()
lp2.getResult()

Optimal
Objective value:  276.0
P(0,0)  =  0.0
P(0,1)  =  0.0
P(0,10)  =  0.0
P(0,11)  =  0.0
P(0,12)  =  0.0
P(0,2)  =  0.0
P(0,3)  =  0.0
P(0,4)  =  0.0
P(0,5)  =  0.0
P(0,6)  =  0.0
P(0,7)  =  0.0
P(0,8)  =  0.0
P(0,9)  =  0.0
P(1,0)  =  0.0
P(1,1)  =  0.0
P(1,10)  =  0.0
P(1,11)  =  0.0
P(1,12)  =  0.0
P(1,2)  =  0.0
P(1,3)  =  0.0
P(1,4)  =  0.0
P(1,5)  =  0.0
P(1,6)  =  0.0
P(1,7)  =  0.0
P(1,8)  =  0.0
P(1,9)  =  0.0
P(10,0)  =  0.0
P(10,1)  =  0.0
P(10,10)  =  0.0
P(10,11)  =  0.0
P(10,12)  =  0.0
P(10,2)  =  0.0
P(10,3)  =  0.0
P(10,4)  =  0.0
P(10,5)  =  1800.0
P(10,6)  =  0.0
P(10,7)  =  0.0
P(10,8)  =  0.0
P(10,9)  =  0.0
P(11,0)  =  5700.0
P(11,1)  =  0.0
P(11,10)  =  0.0
P(11,11)  =  0.0
P(11,12)  =  0.0
P(11,2)  =  0.0
P(11,3)  =  0.0
P(11,4)  =  0.0
P(11,5)  =  0.0
P(11,6)  =  0.0
P(11,7)  =  0.0
P(11,8)  =  0.0
P(11,9)  =  0.0
P(12,0)  =  0.0
P(12,1)  =  0.0
P(12,10)  =  0.0
P(12,11)  =  0.0
P(12,12)  =  0.0
P(12,2)  =  0.0
P(12,3)  =  0.0
P(12,4)  =  0.0
P(12,5)  =  0

## E-N22-K4.vrp

In [0]:
# E-n22-k4.vrp = CE22P

xCoordinates = [145, 151, 159, 130, 128, 163, 146, 161, 142, 163, 148, 128, 156, 129, 146, 164, 141, 147, 164, 129, 155, 139]
yCoordinates = [215, 164, 261, 254, 252, 247, 246, 242, 239, 236, 232, 231, 217, 214, 208, 208, 206, 193, 193, 189, 185, 182]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500, 700]
pickup = [700, 0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500]
capacityOfVehicle = 6000
numberOfVehicles = 7

In [0]:
lp = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)

In [0]:
lp.solve()
lp.getResult()

Optimal
Objective value:  404.9921288065649
P(0,0)  =  0.0
P(0,1)  =  0.0
P(0,10)  =  0.0
P(0,11)  =  0.0
P(0,12)  =  0.0
P(0,13)  =  0.0
P(0,14)  =  0.0
P(0,15)  =  0.0
P(0,16)  =  0.0
P(0,17)  =  0.0
P(0,18)  =  0.0
P(0,19)  =  0.0
P(0,2)  =  0.0
P(0,20)  =  0.0
P(0,21)  =  0.0
P(0,3)  =  0.0
P(0,4)  =  0.0
P(0,5)  =  0.0
P(0,6)  =  0.0
P(0,7)  =  0.0
P(0,8)  =  0.0
P(0,9)  =  0.0
P(1,0)  =  0.0
P(1,1)  =  0.0
P(1,10)  =  0.0
P(1,11)  =  0.0
P(1,12)  =  0.0
P(1,13)  =  0.0
P(1,14)  =  0.0
P(1,15)  =  0.0
P(1,16)  =  0.0
P(1,17)  =  0.0
P(1,18)  =  0.0
P(1,19)  =  0.0
P(1,2)  =  0.0
P(1,20)  =  1900.0
P(1,21)  =  0.0
P(1,3)  =  0.0
P(1,4)  =  0.0
P(1,5)  =  0.0
P(1,6)  =  0.0
P(1,7)  =  0.0
P(1,8)  =  0.0
P(1,9)  =  0.0
P(10,0)  =  5600.0
P(10,1)  =  0.0
P(10,10)  =  0.0
P(10,11)  =  0.0
P(10,12)  =  0.0
P(10,13)  =  0.0
P(10,14)  =  0.0
P(10,15)  =  0.0
P(10,16)  =  0.0
P(10,17)  =  0.0
P(10,18)  =  0.0
P(10,19)  =  0.0
P(10,2)  =  0.0
P(10,20)  =  0.0
P(10,21)  =  0.0
P(10,3)  =  0.